In [ ]:
!pip install mne
!pip install gdown
# !gdown --id 1bc3Oso4uLiv0wUUHmUVFRYc3QdV7lK00
# !gdown --id 1f2DLmnFXLhEt4J0x9ZrtILIMcShR6l42
# !gdown --id 1FCpLa0mq5NcxuWI3ftDjgpJDuBDjkqnY
# !gdown --id 1vSSsh7fwusavI5LnZujw2PsKCKRy40-W
# !gdown --id 1vCk_H45g8crJ5-ScVNhZ7z3noDaeNRKb
# !gdown --id 1TvbSmKFnRhumv3RkS2GuY7e-jVKQPH7_

# !gdown --id 1KC9_02bk0Y_D1E3NDm-Q9qxTi_t4xorD
# !gdown --id 1N9jTbPhxHCv7TXgs6YB18bczkxnPERWv
# !gdown 1vhdJSOb_oVU4B9remQHZSXjy2fjieDN_
# !gdown 1Bvbiu0rplz-uVVOa2jEmOk3RNfV36m4B
!gdown 1j7H0AkYvO-NrcCQ3FDPf4ELyf8q-bMqR
!gdown 1dh2gcxCLULCOwLWXkvEV2C-VeBmxanlA
!gdown 1EpBs6DISLso80kZaxewdq7NnBkqe78-C
!gdown 1diUydgpMGbLlO22L8ZV1iUC5yxNjTxj_

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SeparableConv1D, MaxPooling1D, Dense, Dropout, LSTM, Input, Reshape, Conv2D, BatchNormalization, DepthwiseConv2D, Activation, AveragePooling2D, SeparableConv2D, Flatten,SpatialDropout1D
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mne

In [ ]:
# import tensorflow as tf

# # on Kaggle you must pass tpu="local" here
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")

# # hook up the TPU and bring it online
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

# # now build a TPUStrategy
# strategy = tf.distribute.TPUStrategy(resolver)

# print("Logical TPU devices:", tf.config.list_logical_devices("TPU"))


In [ ]:
# tf.keras.mixed_precision.set_global_policy("float32")

In [ ]:
def EEGNET(filter1=8,D=2):
  filter2=filter1*D;
  model = Sequential()
  model.add(Input(shape = (51,481)))
  model.add(Reshape((51,481,1)))
  model.add(Conv2D(filters = filter1, kernel_size = (1, 80), activation='linear', padding='same'))
  model.add(BatchNormalization())
  model.add(DepthwiseConv2D(kernel_size=(51,1), activation='linear', depth_multiplier=D,padding='valid'))
  model.add(BatchNormalization())
  model.add(Activation('elu'))
  model.add(AveragePooling2D((1,4)))
  model.add(Dropout(0.3))
  model.add(SeparableConv2D(filters = filter2, kernel_size = (1, 16), activation='linear', padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('elu'))
  model.add(AveragePooling2D((1,8)))
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(5, activation='softmax'))
  return model

In [ ]:
def pureLSTM():
  model = Sequential()
  model.add( LSTM(64, return_sequences=True, dropout=0.2,input_shape = (481,51)))
  model.add( LSTM(32, return_sequences=False,dropout=0.2))
  model.add(BatchNormalization())
  model.add( Dropout(0.4))
  model.add( Dense(32, activation='relu'))
  model.add( Dense(5, activation='softmax'))

  return model

In [ ]:
def dscnnLSTM():
  model = Sequential()
  model.add(SeparableConv1D(64,kernel_size=3, input_shape=(481,51), activation='relu',padding='same'))
  model.add(SeparableConv1D(32,kernel_size=3, activation='relu',padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling1D(2))
  model.add(Dense(256,activation='relu'))
  model.add(Dropout(0.6))
  model.add(LSTM(64,return_sequences=True,dropout=0.4))
  model.add(LSTM(64,return_sequences=False,dropout=0.4))
  model.add(BatchNormalization())
  model.add(Dense(32,activation='relu'))
  model.add(Dense(5, activation='softmax'))

  return model

In [ ]:
epochs1 = mne.read_epochs('/kaggle/working/combined_epochsfinal.fif')
X = np.array(mne.concatenate_epochs([epochs1]))
labels1=pd.read_csv("/kaggle/working/Labelsfinal.csv").drop('Unnamed: 0',  axis=1)

In [ ]:
labels1.value_counts()

In [ ]:
Xnew=[]
Ynew=[]
count=0
for i in range(len(labels1['0'])):
    num=labels1['0'][i]
    if num==1 and count >= 1130:
        continue
    else:
        if(num==1):
            Ynew.append([1,0,0,0,0])
            count+=1
        elif(num==2):
            Ynew.append([0,1,0,0,0])
        elif(num==3):
            Ynew.append([0,0,1,0,0])
        elif(num==4):
            Ynew.append([0,0,0,1,0])
        elif(num==5):
            Ynew.append([0,0,0,0,1])
        Xnew.append(X[i])
        

In [ ]:
Xnew=np.array(Xnew)
Ynew=np.array(Ynew)
Xnew.shape, Ynew.shape

In [ ]:
Xnew[0][0].mean(), Xnew[0][0].std()

In [ ]:
for i in range(len(Xnew)):
    for j in range(len(Xnew[i])):
        Xnew[i][j] = (Xnew[i][j] - Xnew[i][j].mean())/Xnew[i][j].std()

In [ ]:
Xnew[0][0].mean(), Xnew[0][0].std()

In [ ]:
X=Xnew
Y=Ynew
X=np.swapaxes(X,axis1=1,axis2=2)
X.shape, Y.shape

In [ ]:
'''from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights to balance the loss function
class_labels = np.array(labels1).T[0]  # Convert one-hot back to integers
classes = np.unique(class_labels)
class_weights = compute_class_weight('balanced', classes=classes, y=class_labels)
class_weight_dict = dict(zip(classes, class_weights))

print("Class weights:", class_weight_dict)'''


In [ ]:
from  sklearn.model_selection import train_test_split
X_train,  X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2)

In [ ]:
import numpy as np

np.isnan(X_train).any(),np.isinf(X_train).any(),np.isnan(Y_train).any(),np.isinf(Y_train).any()


In [ ]:
X_train.shape,Y_train.shape

In [ ]:
# dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
# ValDataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
# dataset = (
#     dataset
#     .shuffle(10_000)
#     .batch(64, drop_remainder=True)
#     .prefetch(tf.data.AUTOTUNE)
# )
# ValDataset = (
#     ValDataset
#     .shuffle(10_000)
#     .batch(64, drop_remainder=True)
#     .prefetch(tf.data.AUTOTUNE)
# )

In [ ]:
# import tensorflow as tf
# import numpy as np

# # TPU init (keep this as you have it)
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.TPUStrategy(resolver)

# # Minimal random data
# x = np.random.randn(512, 32).astype("float32")
# y = np.random.randint(0, 10, (512,)).astype("int32")
# ds = (
#     tf.data.Dataset.from_tensor_slices((x, y))
#       .batch(64, drop_remainder=True)
#       .prefetch(tf.data.AUTOTUNE)
# )

# with strategy.scope():
#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(64, activation="relu", input_shape=(32,)),
#         tf.keras.layers.Dense(10, activation="softmax")
#     ])
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(1e-3),
#         loss="sparse_categorical_crossentropy",
#         metrics=["accuracy"]
#     )

# # Should train cleanly on TPU
# history = model.fit(ds, epochs=5)
# print(history.history)


In [ ]:
pureLSTMmodel= pureLSTM()
opt= Adam(learning_rate=1e-4)
pureLSTMmodel.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
pureLSTMmodel.summary()

In [ ]:
with tf.device('/GPU:0'):
    historyLSTM= pureLSTMmodel.fit(X_train, Y_train, validation_data=(X_test,Y_test), epochs=200,batch_size=32)

In [ ]:
dscnnLSTMmodel= dscnnLSTM()
opt= Adam(learning_rate=1e-4)
dscnnLSTMmodel.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
dscnnLSTMmodel.summary()

In [ ]:
with tf.device('/GPU:0'):
    historydscnn= dscnnLSTMmodel.fit(X_train, Y_train, validation_data=(X_test,Y_test), epochs=200, batch_size=32)

In [ ]:
X=np.swapaxes(X,axis1=1,axis2=2)
EEGNETmodel= EEGNET()
opt= Adam(learning_rate=1e-4)
EEGNETmodel.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
EEGNETmodel.summary()

In [ ]:
with tf.device('/GPU:0'):
    historyEEGNET= EEGNETmodel.fit(X_train, Y_train, validation_data=(X_test,Y_test), epochs=200, batch_size=32)

In [ ]:
X_train.shape

In [ ]:
X=np.swapaxes(X,axis1=1,axis2=2)
np.argmax(dscnnLSTMmodel.predict(X[:500]), axis=1)

In [ ]:
np.argmax(pureLSTMmodel.predict(X[:500]), axis=1)

In [ ]:
X=np.swapaxes(X,axis1=1,axis2=2)
np.argmax(EEGNETmodel.predict(X[:500]), axis=1)

In [ ]:
LSTMloss=historyLSTM.history['loss']
LSTMValLoss=historyLSTM.history['val_loss']
LSTMacc=historyLSTM.history['accuracy']
LSTMValAcc=historyLSTM.history['val_accuracy']
plt.plot(list(range(1,201)),LSTMloss)
plt.plot(list(range(1,201)),LSTMValLoss)
plt.title("Loss Curves for pure LSTM model")

In [ ]:
plt.plot(list(range(1,201)),LSTMacc)
plt.plot(list(range(1,201)),LSTMValAcc)
plt.title("Accuracy Curves for pure LSTM model")

In [ ]:
dscnnLSTMloss=historydscnn.history['loss']
dscnnLSTMValLoss=historydscnn.history['val_loss']
dscnnLSTMacc=historydscnn.history['accuracy']
dscnnLSTMValAcc=historydscnn.history['val_accuracy']
plt.plot(list(range(1,201)),dscnnLSTMloss)
plt.plot(list(range(1,201)),dscnnLSTMValLoss)
plt.title("Loss Curves for DSCNN-LSTM model")

In [ ]:
plt.plot(list(range(1,201)),dscnnLSTMacc)
plt.plot(list(range(1,201)),dscnnLSTMValAcc)
plt.title("Accuracy Curves for DSCNN-LSTM model")

In [ ]:
EEGNETloss=historyEEGNET.history['loss']
EEGNETValLoss=historyEEGNET.history['val_loss']
EEGNETacc=historyEEGNET.history['accuracy']
EEGNETValAcc=historyEEGNET.history['val_accuracy']
plt.plot(list(range(1,201)),EEGNETloss)
plt.plot(list(range(1,201)),EEGNETValLoss)
plt.title("Loss Curves for EEGNET model")
plt.legend(["Training Loss", "Validation Loss"])

In [ ]:
plt.plot(list(range(1,201)),EEGNETacc)
plt.plot(list(range(1,201)),EEGNETValAcc)
plt.title("Accuracy Curves for EEGNET model")
plt.legend(["Training Accuracy", "Validation Accuracy"])